In [ ]:
"""

@author: Eleftherios Trivizakis
@github: github.com/trivizakis

"""

Import custom libraries for data handling & deep learning

In [1]:
from dataset import DataConverter
from data_augmentation import DataAugmentation
from data_generator import DataGenerator
from model import CustomModel as Model
import numpy as np
from utils import Utils
from sklearn.model_selection import StratifiedShuffleSplit
from pathlib import Path
import pickle as pkl
from keras import backend as K

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


Import dataset & labels

In [2]:
hyperparameters = Utils.get_hypes()

my_file = Path(hyperparameters["dataset_dir"] + "labels.pkl")
if not my_file.is_file():
    DataConverter(hyperparameters).convert_png_to_npy()
    
with open(hyperparameters["dataset_dir"]+"labels.pkl", "rb") as file:
    labels = pkl.load(file)

pids = np.load(hyperparameters["dataset_dir"] + "pids.npy")
label_values = np.load(hyperparameters["dataset_dir"] + "labels.npy")

Initialization kfold or hold-out

In [3]:
tst_split_index = 1
val_split_index = 1
#skf_tr_tst = StratifiedKFold(n_splits=hyperparameters["kfold"][0],shuffle=hyperparameters["shuffle"])
skf_tr_val = StratifiedShuffleSplit(n_splits=hyperparameters["kfold"][0], test_size=0.2, train_size=0.8)
values = np.array(list(labels.values()))

Import data, augmentation, model fitting 

In [4]:
#for trval_index, tst_index in skf_tr_tst.split(pids,values):
convergence_pids = pids#[trval_index]
convergence_labels = values#[trval_index]
for tr_index, val_index in skf_tr_val.split(convergence_pids,convergence_labels):
    K.clear_session()
    
    #network version
    version = str(tst_split_index)+"."+str(val_split_index)
    hyperparameters["version"] = "network_version"+version+"\\"
    
    #make dirs        
    Utils.make_dirs(version,hyperparameters)        
    
    #save patient id per network version
    Utils.save_skf_pids(version,convergence_labels[tr_index],convergence_pids[val_index],convergence_pids[val_index],hyperparameters)#pids[tst_index],hyperparameters)        
    
    #offline data augmentation
    if hyperparameters["offline_augmentation"] == True and hyperparameters["data_augmentation"] == True:
        training_set, training_labels = DataConverter(hyperparameters).load_npy_from_hdd(convergence_pids[tr_index], labels)
        training_set, training_labels = DataAugmentation.apply_augmentation(training_set, training_labels,hyperparameters)
        pids_tr, labels_tr=DataConverter(hyperparameters).save_augmented_samples(training_set, training_labels,convergence_pids[tr_index],convergence_labels[tr_index])
    else:
        pids_tr = convergence_pids[tr_index]
        labels_tr = labels
            
    # Generators
    training_generator = DataGenerator(pids_tr, labels_tr, hyperparameters, training=True)
    validation_generator = DataGenerator(convergence_pids[val_index], labels, hyperparameters, training=False)
#    testing_generator = DataGenerator(pids[tst_index], labels, hyperparameters, training=False)
    
    #create network
    cnn = Model.get_model(hyperparameters)
    
    #fit network
    cnn = Model.train_model(cnn,hyperparameters,training_generator,validation_generator)
    Utils.save_hypes(hyperparameters["chkp_dir"]+hyperparameters["version"], "hypes"+version, hyperparameters)
    
    #Update version indeces
    val_split_index+=1
val_split_index=1
tst_split_index+=1

W0922 23:10:12.742325  1220 deprecation_wrapper.py:119] From C:\Users\Lefteris\Anaconda3\envs\deep_learning\lib\site-packages\keras\backend\tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0922 23:10:12.742325  1220 deprecation_wrapper.py:119] From C:\Users\Lefteris\Anaconda3\envs\deep_learning\lib\site-packages\keras\backend\tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0922 23:10:12.757950  1220 deprecation_wrapper.py:119] From C:\Users\Lefteris\Anaconda3\envs\deep_learning\lib\site-packages\keras\backend\tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0922 23:10:12.773574  1220 deprecation_wrapper.py:119] From C:\Users\Lefteris\Anaconda3\envs\deep_learning\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder i

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 150, 150, 64)      1792      
_________________________________________________________________
conv_bn0 (BatchNormalization (None, 150, 150, 64)      256       
_________________________________________________________________
conv0_out (Activation)       (None, 150, 150, 64)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 75, 75, 64)        36928     
_________________________________________________________________
conv_bn1 (BatchNormalization (None, 75, 75, 64)        256       
_________________________________________________________________
conv1_out (Activation)       (None, 75, 75, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 75, 75, 128)       73856     
__________

W0922 23:10:18.562630  1220 deprecation.py:323] From C:\Users\Lefteris\Anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0922 23:10:22.680984  1220 deprecation_wrapper.py:119] From C:\Users\Lefteris\Anaconda3\envs\deep_learning\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0922 23:10:22.680984  1220 deprecation_wrapper.py:119] From C:\Users\Lefteris\Anaconda3\envs\deep_learning\lib\site-packages\keras\callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/2000
133/133 [==============================] - 26s 192ms/step - loss: 1.8085 - acc: 0.6160 - val_loss: 2.2239 - val_acc: 0.5414
Epoch 2/2000
133/133 [==============================] - 21s 160ms/step - loss: 1.4435 - acc: 0.7556 - val_loss: 1.9042 - val_acc: 0.5838
Epoch 3/2000
133/133 [==============================] - 21s 160ms/step - loss: 1.3096 - acc: 0.8073 - val_loss: 1.8790 - val_acc: 0.6434
Epoch 4/2000
133/133 [==============================] - 21s 161ms/step - loss: 1.1774 - acc: 0.8687 - val_loss: 1.3014 - val_acc: 0.8081
Epoch 5/2000
133/133 [==============================] - 22s 163ms/step - loss: 1.0948 - acc: 0.9035 - val_loss: 1.4678 - val_acc: 0.7071
Epoch 6/2000
133/133 [==============================] - 21s 161ms/step - loss: 1.0173 - acc: 0.9368 - val_loss: 1.3695 - val_acc: 0.7444
Epoch 7/2000
133/133 [==============================] - 21s 160ms/step - loss: 0.9322 - acc: 0.9659 - val_loss: 1.1767 - val_acc: 0.8667
Epoch 8/2000
133/133 [===================